In [2]:
%load_ext autoreload
%autoreload 2

import os
import sqlite3
from glob import glob
import sqlite3
import pickle
import pandas as pd
import requests
from arch.univariate.base import ARCHModelResult
from dotenv import load_dotenv
from data import SQLRepository


In [3]:
connection = sqlite3.connect(os.getenv("db_name"), check_same_thread=False)
repo = SQLRepository(connection=connection)

print("repo type:", type(repo))
print("repo.connection type:", type(repo.connection))

repo type: <class 'data.SQLRepository'>
repo.connection type: <class 'sqlite3.Connection'>


In [4]:
from model import GarchModel

# Instantiate a `GarchModel`
gm_ambuja = GarchModel(ticker="AMBUJACEM.BSE", repo=repo, use_new_data=False)

# Does `gm_ambuja` have the correct attributes?
assert gm_ambuja.ticker == "AMBUJACEM.BSE"
assert gm_ambuja.repo == repo
assert not gm_ambuja.use_new_data
assert gm_ambuja.model_directory == os.getenv("model_directory")

In [5]:
# Instantiate `GarchModel`, use new data
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=True)

# Check that model doesn't have `data` attribute yet
assert not hasattr(model_shop, "data")

# Wrangle data
model_shop.wrangle_data(n_observations=1000)

# Does model now have `data` attribute?
assert hasattr(model_shop, "data")

# Is the `data` a Series?
assert isinstance(model_shop.data, pd.Series)

# Is Series correct shape?
assert model_shop.data.shape == (1000,)

model_shop.data.head()

date
2021-12-31    3.795483
2022-01-03   -1.118163
2022-01-04    0.290342
2022-01-05    0.213317
2022-01-06   -1.870154
Name: return, dtype: float64

In [6]:
# Instantiate `GarchModel`, use old data
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=False)

# Wrangle data
model_shop.wrangle_data(n_observations=1000)

# Fit GARCH(1,1) model to data
model_shop.fit(p=1, q=1)

# Does `model_shop` have a `model` attribute now?
assert hasattr(model_shop, "model")

# Is model correct data type?
assert isinstance(model_shop.model, ARCHModelResult)

# Does model have correct parameters?
assert model_shop.model.params.index.tolist() == ["mu", "omega", "alpha[1]", "beta[1]"]

# Check model parameters
model_shop.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2206.55
Distribution:                  Normal   AIC:                           4421.10
Method:            Maximum Likelihood   BIC:                           4440.73
                                        No. Observations:                 1000
Date:                Fri, Jan 16 2026   Df Residuals:                      999
Time:                        12:56:34   Df Model:                            1
                               Mean Model                               
========================================================================
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
mu             0.0333  6.969e-02      0.477      0.633 [ -0.103,  0.170]
                              Volatility Model                             
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
omega          0.7852      1.177      0.667      0.505    [ -1.521,  3.091]
alpha[1]       0.0792  7.995e-02      0.991      0.322 [-7.747e-02,  0.236]
beta[1]        0.7663      0.294      2.604  9.212e-03    [  0.190,  1.343]
===========================================================================

Covariance estimator: robust
"""

In [7]:
# Generate prediction from `model_shop`
prediction = model_shop.predict_volatility(horizon=5)

# Is prediction a dictionary?
assert isinstance(prediction, dict)

# Are keys correct data type?
assert all(isinstance(k, str) for k in prediction.keys())

# Are values correct data type?
assert all(isinstance(v, float) for v in prediction.values())

prediction

{'2026-01-15T00:00:00': 2.433112370990548,
 '2026-01-16T00:00:00': 2.406483536987873,
 '2026-01-19T00:00:00': 2.3837343749776223,
 '2026-01-20T00:00:00': 2.364327135714912,
 '2026-01-21T00:00:00': 2.3477914080209294}

In [10]:
# Save `model_shop` model, assign filename
filename = model_shop.dump()

# Is `filename` a string?
assert isinstance(filename, str)

# Does filename include ticker symbol?
assert model_shop.ticker in filename

# Does file exist?
assert os.path.exists(filename)

filename

'models\\2026-01-16T13-04-13.000183_SHOPERSTOP.BSE.pkl'

In [12]:
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=False)

# Check that new `model_shop_test` doesn't have model attached
assert not hasattr(model_shop, "model")

# Load model
model_shop.load()

# Does `model_shop_test` have model attached?
assert hasattr(model_shop, "model")

model_shop.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2206.55
Distribution:                  Normal   AIC:                           4421.10
Method:            Maximum Likelihood   BIC:                           4440.73
                                        No. Observations:                 1000
Date:                Fri, Jan 16 2026   Df Residuals:                      999
Time:                        12:56:34   Df Model:                            1
                               Mean Model                               
========================================================================
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
mu             0.0333  6.969e-02      0.477      0.633 [ -0.103,  0.170]
                              Volatility Model                             
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
omega          0.7852      1.177      0.667      0.505    [ -1.521,  3.091]
alpha[1]       0.0792  7.995e-02      0.991      0.322 [-7.747e-02,  0.236]
beta[1]        0.7663      0.294      2.604  9.212e-03    [  0.190,  1.343]
===========================================================================

Covariance estimator: robust
"""